In [1]:
import os
import h5py
import numpy as np
import pandas as pd
from scipy.io import loadmat
from scipy.stats import pearsonr

### get the HCP demographics

In [2]:
HCP_u = pd.read_csv('../data/HCP_solar/HCP_s1200_unrestricted.csv', 
                    index_col = 0)

HCP_u.index = HCP_u.index.map(str)

HCP_r = pd.read_csv('../data/HCP_solar/HCP_s1200_restricted.csv', 
                    index_col = 0)

HCP_r.index = HCP_r.index.map(str)


In [3]:
len(HCP_u), len(HCP_r)

(1206, 1206)

### combine S900 Gradients with HCP demogr.

In [4]:
# get HCP - S900 subject list        
subjlist = '../data/subjectListS900_QC_gr.txt'
f = open(subjlist); mylist = f.read().split("\n"); f.close() 
subjlist = joinedlist = mylist[:-1]

len(subjlist)

709

## check if alignment was run correctly

In [5]:
### load group-level FC gradients

ddir = '../data/tout_group/'

# read-in primary and seconardy gradients (G1 & G2) for individual subfields
h1  = h5py.File(os.path.join(ddir, 'Hmean709connGradients_left.h5'), 'r');

G1_fc_LSUB = np.array(h1['gradients_'][0:1024,0])
G1_fc_LCA = np.array(h1['gradients_'][1024:1024+2048,0])
G1_fc_LDG = np.array(h1['gradients_'][1024+2048:1024+2048+1024,0])

G2_fc_LSUB = np.array(h1['gradients_'][0:1024,1])
G2_fc_LCA = np.array(h1['gradients_'][1024:1024+2048,1])
G2_fc_LDG = np.array(h1['gradients_'][1024+2048:1024+2048+1024,1])

### individual LSUB alignment check

In [6]:
gdir = '../data/tout_hippoc_grad'

j = 0 
for subjID in subjlist:
    
    g1_file_LSUB = h5py.File(os.path.join(gdir, 'HCP_'+ subjID + '_G1_%s.h5' % ('LSUB')), 'r' )
    g2_file_LSUB = h5py.File(os.path.join(gdir, 'HCP_'+ subjID + '_G2_%s.h5' % ('LSUB')), 'r' )
    
    g1_LSUB = np.array(g1_file_LSUB['HCP_'+ subjID])  
    g2_LSUB = np.array(g2_file_LSUB['HCP_'+ subjID])  
  
    g1_file_LSUB.close()
    g2_file_LSUB.close()

    r1, p1 = pearsonr(G1_fc_LSUB, g1_LSUB)
    r2, p2 = pearsonr(G2_fc_LSUB, g2_LSUB)

    if r1 < 0 or r2 < 0:
        j += 1
        print(subjID, r1, r2)
        
j

0

### individual LCA alignment check

In [7]:
gdir = '../data/tout_hippoc_grad'

j = 0 
for subjID in subjlist:
    
    g1_file_LCA = h5py.File(os.path.join(gdir, 'HCP_'+ subjID + '_G1_%s.h5' % ('LCA')), 'r' )
    g2_file_LCA = h5py.File(os.path.join(gdir, 'HCP_'+ subjID + '_G2_%s.h5' % ('LCA')), 'r' )
    
    g1_LCA = np.array(g1_file_LCA['HCP_'+ subjID])  
    g2_LCA = np.array(g2_file_LCA['HCP_'+ subjID])  
  
    g1_file_LCA.close()
    g2_file_LCA.close()

    r1, p1 = pearsonr(G1_fc_LCA, g1_LCA)
    r2, p2 = pearsonr(G2_fc_LCA, g2_LCA)

    if r1 < 0 or r2 < 0:
        j += 1
        print(subjID, r1, r2)
        
j

0

### individual LDG alignment check

In [8]:
gdir = '../data/tout_hippoc_grad'

j = 0 
for subjID in subjlist:
    
    g1_file_LDG = h5py.File(os.path.join(gdir, 'HCP_'+ subjID + '_G1_%s.h5' % ('LDG')), 'r' )
    g2_file_LDG = h5py.File(os.path.join(gdir, 'HCP_'+ subjID + '_G2_%s.h5' % ('LDG')), 'r' )
    
    g1_LDG = np.array(g1_file_LDG['HCP_'+ subjID])  
    g2_LDG = np.array(g2_file_LDG['HCP_'+ subjID])  
  
    g1_file_LDG.close()
    g2_file_LDG.close()

    r1, p1 = pearsonr(G1_fc_LDG, g1_LDG)
    r2, p2 = pearsonr(G2_fc_LDG, g2_LDG)

    if r1 < 0 or r2 < 0:
        j += 1
        print(subjID, r1, r2)
        
j

0

## Prepare data frames for solar

### G1, LSUB

In [9]:
mysubjects = []

for subj in subjlist:
    mysubjects.append(subj)
print(len(mysubjects))

tot_node_num_lsub = 1024 # subiculum has 1024 vertices
node_str = []
for i in range(1, tot_node_num_lsub+1):
    node_str.append('node_'+ str(i))
print(len(node_str))
node_str[0], '....', node_str[-1]

mycols = ['age', 'sex'] + node_str 

df = pd.DataFrame(index = mysubjects,
                 columns = mycols)
df.index.name = 'id'

709
1024


In [10]:
gdir = '../data/tout_hippoc_grad'

j = 0 

for subjID in mysubjects:
    
    iA = HCP_r.index.get_loc(subjID)
    iB = HCP_u.index.get_loc(subjID)
    iC = df.index.get_loc(subjID)

    df.iloc[iC]['age'] = HCP_r.iloc[iA]['Age_in_Yrs']
    df.iloc[iC]['sex'] = HCP_u.iloc[iB]['Gender']
    
    
    g1_file_LSUB = h5py.File(os.path.join(gdir, 'HCP_'+ subjID + '_G1_%s.h5' % ('LSUB')), 'r' )
    g1_LSUB = np.array(g1_file_LSUB['HCP_'+ subjID])
    g1_file_LSUB.close()
    df.iloc[iC][node_str] = g1_LSUB

In [11]:
df

,age,sex,node_1,node_2,node_3,node_4,node_5,node_6,node_7,node_8,...,node_1015,node_1016,node_1017,node_1018,node_1019,node_1020,node_1021,node_1022,node_1023,node_1024
id,,,,,,,,,,,,,,,,,,,,,
100206,27,M,0.142592,0.140737,0.147739,0.146034,0.145314,0.145166,0.146482,0.145198,...,-0.0575759,-0.0649671,-0.0628066,-0.0622003,-0.0527981,-0.0644581,-0.0599361,-0.0639843,-0.0628367,-0.0580601
100307,27,F,0.182003,0.18267,0.18066,0.182738,0.182673,0.180661,0.178772,0.166893,...,-0.134107,-0.112374,-0.136159,-0.113298,-0.134755,-0.129645,-0.135579,-0.109167,-0.11372,-0.134851
100408,33,M,0.0479422,0.0447858,0.0698299,0.0554211,0.0563341,0.0424113,0.0735842,0.0600258,...,-0.0386646,-0.033565,-0.0460415,-0.0276705,-0.0456208,-0.0386054,-0.0465238,-0.0272513,-0.0294591,-0.043372
100610,27,M,0.102833,0.105042,0.0966471,0.103025,0.099868,0.103461,0.0921712,0.108893,...,-0.0711061,-0.0702138,-0.065863,-0.0726763,-0.0631246,-0.0703844,-0.0636658,-0.0721315,-0.0730097,-0.0716777
101006,35,F,0.160847,0.152269,0.167646,0.167499,0.162295,0.151889,0.172807,0.150004,...,-0.0783348,-0.0933734,-0.0783592,-0.0890147,-0.0757313,-0.0832705,-0.0758482,-0.0927137,-0.0889727,-0.0787949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990366,22,M,0.0883301,0.109144,0.0792134,0.0832471,0.0831045,0.112673,0.0749225,0.127836,...,-0.0313511,-0.083629,-0.0395023,-0.0822798,-0.0169008,-0.0641333,-0.0220408,-0.0844679,-0.0813522,-0.0372124
991267,30,M,0.176171,0.161722,0.176912,0.174763,0.177276,0.157748,0.176736,0.1505,...,-0.0528742,-0.045728,-0.0465223,-0.038267,-0.0559397,-0.048339,-0.0525152,-0.0395915,-0.042969,-0.052238
992774,35,M,0.137125,0.137295,0.138762,0.138833,0.138111,0.138601,0.141644,0.143453,...,0.0125815,0.00404591,0.000741968,0.00965965,0.00808719,0.0104224,0.000806236,0.00273506,0.0107042,0.0127355


In [12]:
print(len(df))
df = df.dropna()
df.to_csv('../solar/FC_G1_LSUB/G1_LSUB.csv')

709


## G1, LCA 

In [13]:
mysubjects = []

for subj in subjlist:
    mysubjects.append(subj)
print(len(mysubjects))

tot_node_num_lca = 2048 ## CA has 2048 vertices
node_str = []
for i in range(1, tot_node_num_lca+1):
    node_str.append('node_'+ str(i))
print(len(node_str))
node_str[0], '....', node_str[-1]

mycols = []
mycols = ['age', 'sex'] + node_str
len(mycols)

df = pd.DataFrame(index = mysubjects,
                  columns = mycols)
df.index.name = 'id'



709
2048


In [14]:
gdir = '../data/tout_hippoc_grad'

j = 0 

for subjID in mysubjects:
    
    iA = HCP_r.index.get_loc(subjID)
    iB = HCP_u.index.get_loc(subjID)
    iC = df.index.get_loc(subjID)

    df.iloc[iC]['age'] = HCP_r.iloc[iA]['Age_in_Yrs']
    df.iloc[iC]['sex'] = HCP_u.iloc[iB]['Gender']
    
    
    g1_file_LCA = h5py.File(os.path.join(gdir, 'HCP_'+ subjID + '_G1_%s.h5' % ('LCA')), 'r' )
    g1_LCA = np.array(g1_file_LCA['HCP_'+ subjID])
    g1_file_LCA.close()
    df.iloc[iC][node_str] = g1_LCA
    

In [15]:
print(len(df))
df = df.dropna()
df.to_csv('../solar/FC_G1_LCA/G1_LCA.csv')

709


In [16]:
df

,age,sex,node_1,node_2,node_3,node_4,node_5,node_6,node_7,node_8,...,node_2039,node_2040,node_2041,node_2042,node_2043,node_2044,node_2045,node_2046,node_2047,node_2048
id,,,,,,,,,,,,,,,,,,,,,
100206,27,M,0.111416,0.10897,0.0890016,0.0871889,0.117356,0.111247,0.0915516,0.118747,...,-0.0353361,-0.0535251,-0.0551714,-0.0271275,-0.0285859,-0.0376664,-0.0510226,-0.0530432,-0.0548417,-0.043443
100307,27,F,0.119854,0.121932,0.125006,0.120967,0.119366,0.121568,0.125419,0.118292,...,0.00528726,-0.019005,-0.00344468,0.0084717,0.00577915,0.0118915,0.0065547,-0.0131892,-0.00762927,0.0117829
100408,33,M,-0.031658,-0.0306141,-0.0210146,-0.0172283,-0.0179434,-0.0301774,-0.0333814,-0.0192269,...,-0.029658,-0.0400326,-0.0370013,-0.0250529,-0.0268785,-0.0316464,-0.0353891,-0.0399147,-0.0378243,-0.0328052
100610,27,M,0.0704697,0.0661998,0.0478371,0.0455739,0.0846027,0.0707596,0.0461349,0.0845291,...,-0.0626761,-0.0614135,-0.0632925,-0.0608084,-0.0613721,-0.0633849,-0.0646569,-0.0619552,-0.0633889,-0.0644977
101006,35,F,0.120029,0.120841,0.134836,0.138817,0.120524,0.119248,0.132878,0.12058,...,0.0170758,-0.000356968,0.000804793,0.0218042,0.0215921,0.0127001,0.001819,-0.000242141,0.000305722,0.00819015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990366,22,M,0.0204679,0.0303321,0.0931564,0.0942355,-0.0251737,0.0215512,0.0869459,-0.0247622,...,-0.00740714,0.00829527,0.00642873,-0.0078501,-0.00776659,-0.00325924,0.00236941,0.00536024,0.00885649,-0.00147537
991267,30,M,0.0981162,0.10148,0.100212,0.102676,0.0997485,0.0981989,0.0973733,0.0954559,...,-0.057784,-0.0461965,-0.0460037,-0.0588164,-0.058236,-0.0586377,-0.0540199,-0.045617,-0.046226,-0.0588655
992774,35,M,0.0414855,0.0408177,0.0263945,0.0295076,0.0750066,0.0417246,0.0251814,0.0748512,...,-0.00679301,-0.00496677,-0.00462189,-0.0089376,-0.0087742,0.000163804,-0.00476551,-0.00494272,-0.00474813,-0.00266745


### G1, LDG

In [17]:
mysubjects = []

for subj in subjlist:
    mysubjects.append(subj)
print(len(mysubjects))

tot_node_num_ldg = 1024 ## DG has 1024 points 
node_str = []
for i in range(1, tot_node_num_ldg+1):
    node_str.append('node_'+ str(i))
print(len(node_str))
node_str[0], '....', node_str[-1]

mycols = ['age', 'sex'] + node_str 
len(mycols)

df = pd.DataFrame(index = mysubjects,
                 columns = mycols)
df.index.name = 'id'

709
1024


In [18]:
gdir = '../data/tout_hippoc_grad'

j = 0 

for subjID in mysubjects:
    
    iA = HCP_r.index.get_loc(subjID)
    iB = HCP_u.index.get_loc(subjID)
    iC = df.index.get_loc(subjID)

    df.iloc[iC]['age'] = HCP_r.iloc[iA]['Age_in_Yrs']
    df.iloc[iC]['sex'] = HCP_u.iloc[iB]['Gender']
    
    
    g1_file_LDG = h5py.File(os.path.join(gdir, 'HCP_'+ subjID + '_G1_%s.h5' % ('LDG')), 'r' )
    g1_LDG = np.array(g1_file_LDG['HCP_'+ subjID])
    g1_file_LDG.close()
    df.iloc[iC][node_str] = g1_LDG
    


In [19]:
print(len(df))
df = df.dropna()
df.to_csv('../solar/FC_G1_LDG/G1_LDG.csv')

709


### G2, LSUB

In [20]:
mysubjects = []

for subj in subjlist:
    mysubjects.append(subj)
print(len(mysubjects))

tot_node_num_lsub = 1024 # subiculum has 1024 vertices
node_str = []
for i in range(1, tot_node_num_lsub+1):
    node_str.append('node_'+ str(i))
print(len(node_str))
node_str[0], '....', node_str[-1]

mycols = ['age', 'sex'] + node_str 

df = pd.DataFrame(index = mysubjects,
                 columns = mycols)
df.index.name = 'id'

709
1024


In [21]:
gdir = '../data/tout_hippoc_grad'

j = 0 

for subjID in mysubjects:
    
    iA = HCP_r.index.get_loc(subjID)
    iB = HCP_u.index.get_loc(subjID)
    iC = df.index.get_loc(subjID)

    df.iloc[iC]['age'] = HCP_r.iloc[iA]['Age_in_Yrs']
    df.iloc[iC]['sex'] = HCP_u.iloc[iB]['Gender']
    
    
    g2_file_LSUB = h5py.File(os.path.join(gdir, 'HCP_'+ subjID + '_G2_%s.h5' % ('LSUB')), 'r' )
    g2_LSUB = np.array(g2_file_LSUB['HCP_'+ subjID])
    g2_file_LSUB.close()
    df.iloc[iC][node_str] = g2_LSUB

In [22]:
df

,age,sex,node_1,node_2,node_3,node_4,node_5,node_6,node_7,node_8,...,node_1015,node_1016,node_1017,node_1018,node_1019,node_1020,node_1021,node_1022,node_1023,node_1024
id,,,,,,,,,,,,,,,,,,,,,
100206,27,M,0.0865501,0.0515345,0.102094,0.0969066,0.0910128,0.040991,0.121487,0.0258584,...,0.00408603,-0.00285684,0.00381045,-0.0029249,0.0040201,0.000668025,0.00409992,-0.00324143,-0.00280135,0.00371272
100307,27,F,0.0587942,0.043153,0.0634385,0.0621509,0.0613649,0.0369339,0.0642947,0.0261929,...,0.0356436,0.0296608,0.0357355,0.0297838,0.0354791,0.0360639,0.0355141,0.0289504,0.0297873,0.0356998
100408,33,M,0.0625586,0.0551536,0.0676607,0.0674714,0.0676467,0.0527016,0.0658665,0.0563102,...,0.0120837,0.0169598,0.0130537,0.0157842,0.00978536,0.0146389,0.00926017,0.0157707,0.0178584,0.0103541
100610,27,M,0.0965513,0.0928866,0.096414,0.0970537,0.0973238,0.0741427,0.0949116,0.0658303,...,0.0264392,0.029248,0.0271799,0.0284751,0.0284437,0.0259874,0.0259797,0.0296562,0.0285595,0.0267423
101006,35,F,0.106499,0.0813669,0.122453,0.122405,0.120624,0.0711842,0.122821,0.0567537,...,0.0155702,0.0279212,0.0151904,0.0263134,0.0130407,0.0206019,0.0126122,0.0268617,0.0271117,0.0155089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990366,22,M,0.0852874,0.0783281,0.0884874,0.0873328,0.0871693,0.060106,0.0896195,0.0389301,...,-0.0324038,-0.00277672,-0.0249044,-0.00749919,-0.0394202,-0.0114766,-0.0346524,-0.00488654,-0.00385626,-0.0269175
991267,30,M,0.127751,0.0887533,0.131881,0.130503,0.129281,0.0740902,0.133524,0.0490581,...,-0.00344662,0.00702166,-0.0120216,0.00690228,-0.00501327,-0.00475707,-0.00995747,0.00931176,0.00483393,-0.00402859
992774,35,M,0.0885219,0.0759768,0.0913366,0.0924004,0.0904107,0.0656002,0.0909415,0.0542062,...,0.0125504,0.0209986,0.00892199,0.020774,0.00778673,0.0124244,0.00945157,0.0219029,0.018689,0.0110241


In [23]:
print(len(df))
df = df.dropna()
df.to_csv('../solar/FC_G2_LSUB/G2_LSUB.csv')

709


## G2, LCA

In [24]:
mysubjects = []

for subj in subjlist:
    mysubjects.append(subj)
print(len(mysubjects))

tot_node_num_lca = 2048 ## CA has 2048 vertices
node_str = []
for i in range(1, tot_node_num_lca+1):
    node_str.append('node_'+ str(i))
print(len(node_str))
node_str[0], '....', node_str[-1]

mycols = []
mycols = ['age', 'sex'] + node_str
len(mycols)

df = pd.DataFrame(index = mysubjects,
                  columns = mycols)
df.index.name = 'id'


709
2048


In [25]:
gdir = '../data/tout_hippoc_grad'

j = 0 

for subjID in mysubjects:
    
    iA = HCP_r.index.get_loc(subjID)
    iB = HCP_u.index.get_loc(subjID)
    iC = df.index.get_loc(subjID)

    df.iloc[iC]['age'] = HCP_r.iloc[iA]['Age_in_Yrs']
    df.iloc[iC]['sex'] = HCP_u.iloc[iB]['Gender']
    
    
    g2_file_LCA = h5py.File(os.path.join(gdir, 'HCP_'+ subjID + '_G2_%s.h5' % ('LCA')), 'r' )
    g2_LCA = np.array(g2_file_LCA['HCP_'+ subjID])
    g2_file_LCA.close()
    df.iloc[iC][node_str] = g2_LCA
    

In [26]:
print(len(df))
df = df.dropna()
df.to_csv('../solar/FC_G2_LCA/G2_LCA.csv')

709


In [27]:
df

,age,sex,node_1,node_2,node_3,node_4,node_5,node_6,node_7,node_8,...,node_2039,node_2040,node_2041,node_2042,node_2043,node_2044,node_2045,node_2046,node_2047,node_2048
id,,,,,,,,,,,,,,,,,,,,,
100206,27,M,0.0782927,0.0710081,0.0155358,0.00667688,0.0983033,0.0774251,0.0281557,0.0983247,...,-0.0328461,-0.0263229,-0.0267427,-0.035487,-0.0348935,-0.0328015,-0.0283267,-0.0273184,-0.0267019,-0.030257
100307,27,F,0.0515495,0.0486528,0.0300438,0.0266721,0.0650139,0.052535,0.0311405,0.0658584,...,-0.0186892,-0.0176108,-0.0216799,-0.0177776,-0.0187048,-0.0207485,-0.0224006,-0.0184451,-0.0196845,-0.0207133
100408,33,M,0.0403744,0.0330825,0.00713896,0.00686675,0.0633926,0.0367474,0.0236974,0.0698579,...,-0.0213854,-0.0245845,-0.0288479,-0.0208789,-0.0215099,-0.0256545,-0.0277431,-0.0260408,-0.0278187,-0.0285338
100610,27,M,0.0516071,0.0453754,0.0296303,0.0247512,0.0540672,0.0517157,0.0305198,0.0537509,...,-0.0164763,-0.00901856,-0.0128431,-0.0191049,-0.0183614,-0.0158889,-0.016152,-0.0114648,-0.0123637,-0.0162273
101006,35,F,0.0549953,0.0492887,0.0201008,0.0162013,0.0793194,0.0540959,0.0246454,0.0793295,...,-0.0182363,-0.0120438,-0.0133059,-0.0194728,-0.0191268,-0.0165234,-0.0148311,-0.0126178,-0.0129579,-0.0150924
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990366,22,M,0.0500584,0.0386983,-0.0214272,-0.0222743,0.0976555,0.0496138,-0.0150678,0.0971354,...,-0.0493962,-0.0491132,-0.0514421,-0.0484504,-0.0489072,-0.0499617,-0.0552532,-0.0479267,-0.0515213,-0.0517925
991267,30,M,0.0251593,0.0182563,0.00153292,-0.000408272,0.058763,0.0247616,0.00341867,0.0620304,...,0.0113178,0.00843709,0.00802291,0.0125039,0.0121086,0.0124287,0.0104744,0.00838613,0.00915595,0.0133921
992774,35,M,0.0205663,0.0162016,-0.0146774,-0.0156932,0.0756455,0.0214704,-0.00732925,0.0788828,...,-0.00736671,-0.00734277,-0.00719503,-0.00341273,-0.00346116,-0.00582193,-0.0061745,-0.00737387,-0.0072806,-0.00750013


## G2, LDG

In [28]:
mysubjects = []

for subj in subjlist:
    mysubjects.append(subj)
print(len(mysubjects))

tot_node_num_lsub = 1024 
node_str = []
for i in range(1, tot_node_num_lsub+1):
    node_str.append('node_'+ str(i))
print(len(node_str))
node_str[0], '....', node_str[-1]

mycols = ['age', 'sex'] + node_str 

df = pd.DataFrame(index = mysubjects,
                 columns = mycols)
df.index.name = 'id'

709
1024


In [29]:
gdir = '../data/tout_hippoc_grad'

j = 0 

for subjID in mysubjects:
    
    iA = HCP_r.index.get_loc(subjID)
    iB = HCP_u.index.get_loc(subjID)
    iC = df.index.get_loc(subjID)

    df.iloc[iC]['age'] = HCP_r.iloc[iA]['Age_in_Yrs']
    df.iloc[iC]['sex'] = HCP_u.iloc[iB]['Gender']
    
    
    g2_file_LDG = h5py.File(os.path.join(gdir, 'HCP_'+ subjID + '_G2_%s.h5' % ('LDG')), 'r' )
    g2_LDG = np.array(g2_file_LDG['HCP_'+ subjID])
    g2_file_LDG.close()
    df.iloc[iC][node_str] = g2_LDG
    


In [30]:
print(len(df))
df = df.dropna()
df.to_csv('../solar/FC_G2_LDG/G2_LDG.csv')

709
